## `GPUDrive` simulator concepts

In this notebook, we demonstrate how to work with the `GPUDrive` simulator and access its basic attributes in Python. The simulator, written in C++, is built on top of the [Madrona Engine](https://madrona-engine.github.io/).

In [1]:
import os
import torch
from pathlib import Path
import madrona_gpudrive

# Set working directory to the base directory 'gpudrive_madrona'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive_madrona' not found")
os.chdir(working_dir)

### Summary

- `GPUDrive` simulations are discretized traffic scenarios. A scenario is a constructed snapshot of traffic situation at a particular timepoint.
- The state of the vehicle of focus is referred to as the **ego state**. Each vehicle has their own partial view of the traffic scene; and a visible state is constructed by parameterizing the view distance of the driver. 
- The `step()` method advances the simulation with a desired step size. By default, the dynamics of vehicles are driven by a kinematic bicycle model. If a vehicle is not controlled (that is, we do not give it actions), its position, heading, and speed will be updated according to a the log replay demonstrations.


### Instantiating a sim object with default parameters

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
params = madrona_gpudrive.Parameters()
params.polylineReductionThreshold = 0.1

sim = madrona_gpudrive.SimManager(
    exec_mode=madrona_gpudrive.madrona.ExecMode.CUDA
    if device == "cuda"
    else madrona_gpudrive.madrona.ExecMode.CPU,
    gpu_id=0,
    scenes=["data/processed/examples/tfrecord-00000-of-01000_325.json"],
    params=params,  # Environment parameters
)

The simulator provides the following functions:
- `reset(world_idx)` resets a specific world or environment at the given index.

In [4]:
sim.reset([0])

- `step()` advances the dynamics of all worlds.

In [5]:
sim.step()

### Exporting tensors

To retrieve a tensor from the simulator, call the specific `tensor()` method, followed by either `to_torch()` or `to_jax()`.

For example, here is how to access the ego state, or self-observation tensor:

In [6]:
observation_tensor = sim.self_observation_tensor().to_torch()

observation_tensor.shape, observation_tensor.device

(torch.Size([1, 64, 8]), device(type='cpu'))

Or alternatively:

In [ ]:
observation_tensor_jax = sim.self_observation_tensor().to_jax()

observation_tensor_jax.shape, observation_tensor_jax.devices()

Here are all available tensor exports and methods on the sim object:

In [8]:
for attr in dir(sim):
    if not attr.startswith("_"):
        print(attr)

absolute_self_observation_tensor
action_tensor
agent_roadmap_tensor
controlled_state_tensor
deleteAgents
deleted_agents_tensor
depth_tensor
done_tensor
expert_trajectory_tensor
info_tensor
lidar_tensor
map_name_tensor
map_observation_tensor
metadata_tensor
partner_observations_tensor
reset
response_type_tensor
reward_tensor
rgb_tensor
self_observation_tensor
set_maps
shape_tensor
step
steps_remaining_tensor
valid_state_tensor
world_means_tensor


### Inspect valid and controlled agents

To check the number of agents and road points in each world, you can use the `shape_tensor`.

The shape tensor is a 2D tensor where the first dimension represents the number of worlds, and the second dimension represents the shape of each world.

In [ ]:
shape_tensor = sim.shape_tensor().to_jax()
print(f"Shape tensor has a shape of (Num Worlds, 2): {shape_tensor.shape}")

for world_idx in range(shape_tensor.shape[0]):
    print(
        f"World {world_idx} has {shape_tensor[world_idx][0]} VALID agents and {shape_tensor[world_idx][1]} VALID road objects"
    )

The number of **valid** agents indicates the number of controllable agents (vehicles). Some vehicles or bicycles may be initialized in incorrect positions or remain static; these are marked as **invalid** and cannot be controlled.

The sim comes with a mask that indicates which agents can be controlled. Entries are `1` for agents that can be controlled, and `0` otherwise.

In [10]:
controlled_state_tensor = sim.controlled_state_tensor().to_torch()
print(
    "Controlled state tensor has a shape of (num_worlds, max_num_agents_in_scene, 1): ",
    controlled_state_tensor.shape,
)

Controlled state tensor has a shape of (num_worlds, max_num_agents_in_scene, 1):  torch.Size([1, 64, 1])


In [11]:
# We can control 3 agents in this world
controlled_state_tensor.squeeze()

tensor([1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)

In [12]:
controlled_state_tensor.sum().item()

5

### Actions

The action space is determined by the chosen dynamics_model. By default, it uses a bicycle model with the following tuple of actions:

- **Acceleration**: Continuous float values representing the acceleration applied to the agents. This affects how quickly an agent speeds up or slows down.
- **Steering Angle**: Continuous float values representing the steering angle, following the bicycle kinematic model. This determines how sharply an agent turns.
- **Heading Angle**: Continuous float values for the heading angle, which control the direction an agent is facing.


The action tensor stores the current actions for all agents across all worlds:

In [13]:
action_tensor = sim.action_tensor().to_torch()
print(
    f"Action tensor has a shape of (num_worlds, max_num_agents_in_scene, 3): {action_tensor.shape}"
)

Action tensor has a shape of (num_worlds, max_num_agents_in_scene, 3): torch.Size([1, 64, 10])


To set the actions for all controlled agents, we use the `copy_()` method:

In [14]:
actions_tensor = sim.action_tensor().to_torch()

actions = torch.full(actions_tensor.shape, 1.0)
actions_tensor.copy_(actions)

print(f"Actions tensor after setting all actions to 1: {actions_tensor[0][0]}")

# Call step() to apply the actions
sim.step()

Actions tensor after setting all actions to 1: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


### Inspecting the simulator settings

In [17]:
params = madrona_gpudrive.Parameters()

print("Parameters:")
for attr in dir(params):
    if not attr.startswith("__"):
        value = getattr(params, attr)
        print(f"{attr:20}: {value}")
        if attr == "rewardParams":
            print("Reward parameters:")
            reward_params = getattr(params, attr)
            for attr2 in dir(reward_params):
                if not attr2.startswith("__"):
                    value2 = getattr(reward_params, attr2)
                    print(f"    {attr2:18}: {value2}")

Parameters:
IgnoreNonVehicles   : False
collisionBehaviour  : madrona_gpudrive.CollisionBehaviour.AgentStop
disableClassicalObs : False
dynamicsModel       : madrona_gpudrive.DynamicsModel.Classic
enableLidar         : False
initOnlyValidAgentsAtFirstStep: True
isStaticAgentControlled: False
maxNumControlledAgents: 10000
observationRadius   : 0.0
polylineReductionThreshold: 0.0
rewardParams        : <madrona_gpudrive.RewardParams object at 0x7f9d1c273ab0>
Reward parameters:
    distanceToExpertThreshold: 0.0
    distanceToGoalThreshold: 0.0
    rewardType        : madrona_gpudrive.RewardType.DistanceBased
roadObservationAlgorithm: madrona_gpudrive.FindRoadObservationsWith.KNearestEntitiesWithRadiusFiltering


### Configuring the simulator 

To set the parameters of the simulator, fill in the values for each attribute of the parameter object as below. This allows you to customize the simulation settings.

The params object can be passed to the sim constructor like this:

```Python
sim = gpudrive_madrona.SimManager(
    ...
    params=params 
)
```

In [18]:
reward_params = madrona_gpudrive.RewardParams()
reward_params.rewardType = madrona_gpudrive.RewardType.DistanceBased
reward_params.distanceToGoalThreshold = 1.0
reward_params.distanceToExpertThreshold = 1.0

# Initialize Parameters
params = madrona_gpudrive.Parameters()
params.polylineReductionThreshold = 1.0
params.observationRadius = 100.0
params.collisionBehaviour = madrona_gpudrive.CollisionBehaviour.Ignore
params.maxNumControlledAgents = 10
params.rewardParams = reward_params

### Step simulator and access information



In [19]:
actions_shape = sim.action_tensor().to_torch().shape
dones = sim.done_tensor().to_torch()


obs, rews, dones = (
    sim.self_observation_tensor().to_torch(),
    sim.reward_tensor().to_torch(),
    sim.done_tensor().to_torch(),
)
actions = torch.rand(actions_shape)
sim.action_tensor().to_torch().copy_(actions)
sim.step()

Here’s your text with all the headings formatted as lowercase (except for the first letter):

---

## Simulator configurations and objects

This outlines the key components of the simulator, including observation tensors, rewards, road reduction algorithms, collision behavior, and miscellaneous parameters.

### Observation space

- You can use the datatypes in [`gpudrive/datatypes/observations.py`](https://github.com/Emerge-Lab/gpudrive/blob/main/pygpudrive/datatypes/observation.py) to easily index the tensors below. More documentation is provided there.

#### **SelfObservation**

The `SelfObservation` tensor of shape `(6,)` for each agent represents the agent's state. The respective values are:

- `SelfObservation[0]`: Current *speed* of the agent.
- `SelfObservation[1]`: *Length* of the agent.
- `SelfObservation[2]`: *Width* of the agent.
- `SelfObservation[3:4]`: *Coordinates (x, y)* of the goal relative to the agent.
- `SelfObservation[5]`: Collision state of the agent (`0` for no collision, `1` for collision).

#### **MapObservation**

The `MapObservation` tensor of shape `(8,)` provides absolute position data for map objects. Values include:

- `MapObservation[0:1]`: Position `(x, y)` of the map object.
- `MapObservation[2:4]`: Scale `(length, width, height)` of the map object.
- `MapObservation[5]`: Heading angle of the map object.
- `MapObservation[6]`: Type of the map object.
- `MapObservation[7]`: Road ID (`>0` for valid ID, `-1` for unassigned, `0` for missing data).

#### **PartnerObservation**

The `PartnerObservation` tensor of shape `(num_agents-1, 7)` provides data about other agents relative to the ego agent within the `observationRadius`:

- `PartnerObservation[0]`: *Speed* of the neighboring agent.
- `PartnerObservation[1:2]`: *Relative position (x, y)* of the neighboring agent.
- `PartnerObservation[3]`: *Orientation* of the neighboring agent.
- `PartnerObservation[4:5]`: *Dimensions (length, width)* of the neighboring agent.
- `PartnerObservation[6]`: Type of the neighboring agent.

#### **AgentMapObservations**

The `AgentMapObservations` tensor of shape `(num_road_objs, 8)` provides road object data relative to the ego agent. The values are identical to `MapObservation`.

### Rewards

- **Reward types**:
  - `DistanceBased`: Distance from the agent to the goal.
  - `OnGoalAchieved`: Binary reward (`1` for reaching goal, `0` otherwise) (recommended).
  - `Dense`: Distance from the expert trajectory.
- **Thresholds**:
  - `distanceToGoalThreshold`: Default: `0.0`.
  - `distanceToExpertThreshold`: Default: `0.0`.

### Road graph 

- You can use the datatypes in [`gpudrive/datatypes/roadgraph.py`](https://github.com/Emerge-Lab/gpudrive/blob/main/gpudrive/datatypes/roadgraph.py) to easily index the tensors below. More documentation is provided there.

#### Road reduction algorithm

The **Visvalingam-Whyatt Algorithm** is applied to simplify road lines and edges.

- **Parameter**:  
  `polylineReductionThreshold`: Ranges from `0` (no reduction) to `1`. Default: `0.5`.

### Collision behavior

Three options for the collision behavior are available. These determine what happens when an agent touches a road edge or other agents' bounding box.

- `AgentStop`: Agents stop upon collision.
- `AgentRemoved`: Agents are removed from the scene upon collision.
- `Ignore`: Collision is logged but does not affect the agent's movement.

### Miscellaneous parameters

- `ObservationRadius`: Radius around every agent that determines visible view. Elements outside this radius are zeroed out.
- `MaxNumControlledVehicles`: Maximum number of controlled agents. Default: First valid agents are controlled.
- `IgnoreNonVehicles`: Excludes non-vehicle entities (e.g., pedestrians, cyclists). Default: `false`.
- `roadObservationAlgorithm`: Options:
  - `KNearestEntitiesWithRadiusFiltering`: Limits observations to the nearest points within `observationRadius`.
  - `AllEntitiesWithRadiusFiltering`: Searches all points within `observationRadius`. Default: `KNearestEntitiesWithRadiusFiltering`.
- `initOnlyValidAgentsAtFirstStep`: Initializes only valid agents. Default: `true`.
- `isStaticAgentControlled`: Controls if parked agents can be set static. Default: `false`.
- `enableLidar`: Enables lidar observations.
- `disableClassicalObs`: Disables `PartnerObservations` and `AgentMapObservations` for performance. Default: `false`.

### Object types

**Road types**:
- `RoadEdge`, `RoadLine`, `RoadLane`, `CrossWalk`, `SpeedBump`, `StopSign`.

**Agent types**:
- `Vehicle`, `Pedestrian`, `Cyclist`.

**Other types**:
- `Padding`: Ensures consistent output shapes.
- `None`: Marks entities as invalid (e.g., out of `ObservationRadius` or removed due to collision).

For any further details or indexing references, please explore [`gpudrive/datatypes`](https://github.com/Emerge-Lab/gpudrive/tree/main/gpudrive/datatypes) module.

### Other exported tensors

- By default, the x, y, z coordinates are de-meaned to center everything at 0. To revert back to the original coordinates, we export the `world_means_tensor`. 
    - Shape:  (`num_worlds`, 3)